In [1]:
import pandas as pd
from bs4 import BeautifulSoup as bs
import requests
import datetime


In [16]:
daily = "https://www.tn.gov/health/cedep/ncov.html"
viral = pd.read_html(daily)
df=viral[3]
testing = df.drop([10])
age_groups = testing.rename(columns={'Age Ranges of Confirmed Cases.1': 'Number_Cases'})
age_groups


,Age Ranges of Confirmed Cases,Number_Cases
0,0-10,36
1,11-20,138
2,21-30,665
3,31-40,456
4,41-50,437
5,51-60,484
6,61-70,331
7,71-80,184
8,80+,97
9,Pending,17


In [39]:
daily = "https://www.tn.gov/health/cedep/ncov.html"
viral = pd.read_html(daily)
df2 = viral[5]
TN_county = df2[:-3]
TN_county["County"]=TN_county["County"].str.split(" County", expand = True)



C:\Users\clayf\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,County,Positive,Negative,Death
0,Anderson,10,164,NaN
1,Bedford,6,110,NaN
2,Benton,4,36,NaN
3,Bledsoe,2,15,NaN
4,Blount,32,148,NaN
...,...,...,...,...
90,Wayne,2,31,NaN
91,Weakley,1,68,NaN
92,White,2,77,NaN
93,Williamson,199,1434,3.0


In [40]:
TNpop = pd.read_csv("Population Estimates by County.csv")
TNpop= TNpop.drop([96])
TNpop = TNpop.drop([0])
TNpop["Population Estimates by County"]= TNpop["Population Estimates by County"].str.split(" County", n = 1, expand = True)
TNpop = TNpop.rename(columns={'Population Estimates by County': 'County'})
TN_county_populations = TNpop.rename(columns={'Unnamed: 1': 'Population'})
TN_county_populations["Population"] = TN_county_populations["Population"].str.replace(",","").astype(int)
TN_county_populations


,County,Population
1,Anderson,76978
2,Bedford,49713
3,Benton,16160
4,Bledsoe,15064
5,Blount,133088
...,...,...
91,Wayne,16673
92,Weakley,33328
93,White,27345
94,Williamson,238412


In [ ]:

data=viral[4]
data

In [41]:
TN_data = TN_county_populations.merge(TN_county)
TN_data


,County,Population,Positive,Negative,Death
0,Anderson,76978,10,164,NaN
1,Bedford,49713,6,110,NaN
2,Benton,16160,4,36,NaN
3,Bledsoe,15064,2,15,NaN
4,Blount,133088,32,148,NaN
...,...,...,...,...,...
88,Wayne,16673,2,31,NaN
89,Weakley,33328,1,68,NaN
90,White,27345,2,77,NaN
91,Williamson,238412,199,1434,3.0


In [42]:
TN_data["Percentage of County Population"] = round((TN_data["Positive"]/TN_data["Population"])*100,3)
TN_data.reset_index(drop=True)
TN_data["Percent_Tested"] = round(((TN_data["Positive"]+TN_data["Negative"])/TN_data["Population"])*100,1)
TN_data=TN_data.sort_values("Percent_Tested", ascending=False)
TN_data=TN_data
TN_data


,County,Population,Positive,Negative,Death,Percentage of County Population,Percent_Tested
41,Houston,8201,1,77,NaN,0.012,1.0
18,Davidson,694144,617,4813,6.0,0.089,0.8
91,Williamson,238412,199,1434,3.0,0.083,0.7
92,Wilson,144657,62,756,NaN,0.043,0.6
80,Sumner,191283,268,913,7.0,0.140,0.6
...,...,...,...,...,...,...,...
45,Johnson,17788,2,7,NaN,0.011,0.1
29,Greene,69069,16,87,1.0,0.023,0.1
67,Polk,16832,1,19,NaN,0.006,0.1
33,Hancock,6620,0,6,NaN,0.000,0.1


In [14]:
url2="https://raw.githubusercontent.com/nytimes/covid-19-data/master/us-counties.csv"
viral_times=pd.read_csv(url2)
TN=viral_times[viral_times["state"] == 'Tennessee']
# TN=TN[TN["date"] == '2020-03-31']
TN=TN.rename(columns={'county':'County'})
TN=TN.rename(columns={'date':'Date'})
TN=TN.rename(columns={'cases':'Positive'})
TN=TN.rename(columns={'deaths':'Deaths'})
TN=TN.drop(['state'], axis=1)
TN

,Date,County,fips,Positive,Deaths
570,2020-03-05,Williamson,47187.0,1,0
639,2020-03-06,Williamson,47187.0,1,0
731,2020-03-07,Williamson,47187.0,1,0
841,2020-03-08,Davidson,47037.0,1,0
842,2020-03-08,Shelby,47157.0,1,0
...,...,...,...,...,...
23582,2020-03-31,Wayne,47181.0,1,0
23583,2020-03-31,Weakley,47183.0,1,0
23584,2020-03-31,White,47185.0,1,0
23585,2020-03-31,Williamson,47187.0,111,0


In [15]:
TN_deaths = TN_data.merge(TN,left_on='County',right_on='County', how='left')
TN_deaths.head(60)

,County,Population,Positive_x,Negative,Death,Percentage of County Population,Percent_Tested,Date,fips,Positive_y,Deaths
0,Houston,8201,1.0,70.0,NaN,0.012,0.9,2020-03-22,47083.0,1.0,0.0
1,Houston,8201,1.0,70.0,NaN,0.012,0.9,2020-03-23,47083.0,1.0,0.0
2,Houston,8201,1.0,70.0,NaN,0.012,0.9,2020-03-24,47083.0,1.0,0.0
3,Houston,8201,1.0,70.0,NaN,0.012,0.9,2020-03-25,47083.0,3.0,0.0
4,Houston,8201,1.0,70.0,NaN,0.012,0.9,2020-03-26,47083.0,2.0,0.0
5,Houston,8201,1.0,70.0,NaN,0.012,0.9,2020-03-27,47083.0,2.0,0.0
6,Houston,8201,1.0,70.0,NaN,0.012,0.9,2020-03-28,47083.0,2.0,0.0
7,Houston,8201,1.0,70.0,NaN,0.012,0.9,2020-03-29,47083.0,2.0,0.0
8,Houston,8201,1.0,70.0,NaN,0.012,0.9,2020-03-30,47083.0,1.0,0.0
9,Houston,8201,1.0,70.0,NaN,0.012,0.9,2020-03-31,47083.0,1.0,0.0


df_counties=viral[4]
file = datetime.datetime.today()
date = file[0:10]
date


IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices

In [ ]:
# from pathlib import Path
# p = Path('C:/Users/clayf/Documents/HOC/Covid_predictions/TN/Resources/')
df_counties.to_csv('C:/Users/clayf/Documents/HOC/Covid_predictions/TN/Resources/' + date +'counties2.csv',index=False)

In [ ]:
US = "https://www.cdc.gov/coronavirus/2019-ncov/cases-updates/cases-in-us.html?CDC_AA_refVal=https%3A%2F%2Fwww.cdc.gov%2Fcoronavirus%2F2019-ncov%2Fcases-in-us.html"
us_data = pd.read_html(US)
us_total_cases = us_data[0]
us_total_cases.rename(columns={list(us_total_cases)[1]:'Total'}, inplace=True)
us_total_cases['Total']=us_total_cases['Total'].astype(int)
US = us_total_cases.iloc[3,1]

In [ ]:
date = datetime.datetime.today()
date_modify = str(date)
date_to_add = date_modify[0:10]
x['date'] = date_to_add



In [ ]:
TN_ratio = (TN/6700000)*100
US_ratio = (US/372000000)*100

print(TN_ratio)
print(US_ratio)